# CSV Analyzer


Generative Question and Answer (QnA) analyzer of csv files. The focus is on small csv files (100s or 1000s of rows) where traditionally users are doing analysis on a spreadsheet but want a more interactive way to do their analysis. 

Today you can drop a csv on your LLM of choice and can get answers to some complex questions that focus on language summarization tasks like topic categorization, sentiment analysis, sampling of data. Where LLM usually fail is in doing computation of your data i.e. filtering, aggregation, counts etc. 

In order to overcome this drawback we are taking advantage of an agentic approach while giving it access to a pandas runner. This allows the agent the ability to run computations via pandas commands. The agent will use a reAct framework to be able to think through the problem and access the pandas dataframe only when necessary. 

# SetUp

Install the Langchain packages: Open a terminal window and enter the command below

In [22]:
#!pip install --upgrade google-cloud-aiplatform
!pip install vertexai
!pip install langchain-google-genai
!pip install langchain
!pip install langchain-experimental
!pip install --no-deps langchain-google-vertexai 
!pip install panel

^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user
^C
Traceback (most recent call last):
  File "/opt/conda/bin/pip", line 6, in <module>
    from pip._internal.cli.main import main
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/build_env.py", line 18, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_

In [18]:
!pip install panel

  Using cached bokeh-3.6.0-py3-none-any.whl.metadata (12 kB)
Using cached bokeh-3.6.0-py3-none-any.whl (6.9 MB)


In [2]:
!pip --version

pip 24.2 from /opt/conda/lib/python3.10/site-packages/pip (python 3.10)


Initialize variables

In [7]:
PROJECT = 'felipe-sandbox-354619' # replace with project ID
BUCKET = PROJECT + "-bucket" 
CSV_FILE = "files/customer-comments.csv"

Colab setup if needed

In [8]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


## Langchain Setup

### Tools

In [54]:
import pandas as pd
import json
from langchain.agents import Tool
from langchain.tools import tool
from io import StringIO
from pydantic import BaseModel, Field


@tool("pandas_calculator", return_direct=False)
def pandas_calculator(dictionary:str):
    """
    Evaluates an expression on a Pandas DataFrame and returns the result.
    Inputs include an expression expr and a file path csv_file_path
    Inputs should be in format {\"expr\": str, \"csv_file_path\": str}
    """
    dictionary = json.loads(dictionary)
    csv_file_path = dictionary["csv_file_path"]
    expr = dictionary["expr"]
    
    try:
        df = pd.read_csv(csv_file_path)
        print(df)
        # evaluate expr on the pandas dataframe
        result = eval(expr)
        # return the result
        return str(result)
    except Exception as e:
        return f"Error: Could not load data or evaluate expression. {e}"

In [55]:
@tool("csv_summary_statistics", return_direct=False)
def csv_summary_statistics(dictionary:str):
    """
    Gets summary statistics on the csv to understand the data
    Before running pandas calculator please run the statistics first
    Inputs should be in format {\"csv_input_file\": str}
    """
    
    print(dictionary)
    dictionary = json.loads(dictionary)
    input_path = dictionary["csv_input_file"]
    
    try:
        df = pd.read_csv(input_path)
        # evaluate expr on the pandas dataframe
        result = "sample data: \n" + df.head().to_string() + "\n\n"
        #result += "information on the data: " + df.info().to_string()
        #print(df.info())
        # return the result
        return str(result)
    except Exception as e:
        return f"Error: Could not load data or evaluate expression. {e}"
    
test_path = "{\"csv_input_file\":\"" + CSV_FILE + "\"}"
print(test_path)
csv_summary_statistics(test_path)

{"csv_input_file":"files/customer-comments.csv"}
{"csv_input_file":"files/customer-comments.csv"}


/var/tmp/ipykernel_4240/992228356.py:26: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  csv_summary_statistics(test_path)


"sample data: \n   index                                       comment country\n0      1                  The product arrived damaged.      US\n1      2              I'm very happy with the service!  Mexico\n2      3             I have a question about my order.  Canada\n3      4                        The delivery was late.      US\n4      5  The customer support agent was very helpful.  Canada\n\n"

In [56]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_google_vertexai import ChatVertexAI
from langchain.memory import ConversationBufferWindowMemory


llm = ChatVertexAI(model="gemini-1.5-pro-002")
prompt = hub.pull("hwchase17/react-chat")
chat_history = ConversationBufferWindowMemory(k=10)

agent = create_react_agent(llm, [pandas_calculator,csv_summary_statistics], prompt)

agent_executor = AgentExecutor(agent=agent,
                               tools=[pandas_calculator,csv_summary_statistics],
                               verbose=True, handle_parsing_errors=True,
                               max_iterations=5)

def append_chat_history(input, response):
    chat_history.save_context({"input": input}, {"output": response})

def invoke(input):
    if (input):
        print("input: " + str(input))
        msg = {
            "input": input,
            "chat_history": chat_history.load_memory_variables({}),
        }
        print(f"Input: {msg}")

        response = agent_executor.invoke(msg)
        print(f"Response: {response}")

        append_chat_history(response["input"], response["output"])
        print(f"History: {chat_history.load_memory_variables({})}")

        return response["output"]
    else:
        return ""

/opt/conda/lib/python3.10/site-packages/langsmith/client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [57]:
invoke("Hi")
invoke(CSV_FILE + "\n\n how many rows are in this file?")

input: Hi
Input: {'input': 'Hi', 'chat_history': {'history': ''}}


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Hi there! How can I help you today?
! How can I help you today?


> Finished chain.
Response: {'input': 'Hi', 'chat_history': {'history': ''}, 'output': 'Hi there! How can I help you today?\n! How can I help you today?'}
History: {'history': 'Human: Hi\nAI: Hi there! How can I help you today?\n! How can I help you today?'}
input: files/customer-comments.csv

 how many rows are in this file?
Input: {'input': 'files/customer-comments.csv\n\n how many rows are in this file?', 'chat_history': {'history': 'Human: Hi\nAI: Hi there! How can I help you today?\n! How can I help you today?'}}


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: csv_summary_statistics
Action Input: {"csv_input_file": "files/customer-comments.csv"}{"csv_input_file": "files/customer-comments.csv"}
sample data: 
   index         

'There are 100 rows in the file.\n00 rows in the file.'

In [9]:
invoke("can you count how many rows are from the US?")

Input: {'input': 'can you count how many rows are from the US?', 'chat_history': {'history': 'Human: Hi\nAI: Hello! How can I help you today?\n How can I help you today?\nHuman: files/customer-comments.csv\n\n how many rows are in this file?\nAI: There are 100 rows in the file.\n00 rows in the file.'}}


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: csv_summary_statistics
Action Input: {"csv_input_file": "files/customer-comments.csv"}{"csv_input_file": "files/customer-comments.csv"}
sample data: 
   index                                       comment country
0      1                  The product arrived damaged.      US
1      2              I'm very happy with the service!  Mexico
2      3             I have a question about my order.  Canada
3      4                        The delivery was late.      US
4      5  The customer support agent was very helpful.  Canada

Do I need to use a tool? Yes
Action: pandas_calculator
Action Input: {"expr": "df

In [10]:
invoke("how many rows include the word product?")

Input: {'input': 'how many rows include the word product?', 'chat_history': {'history': 'Human: Hi\nAI: Hello! How can I help you today?\n How can I help you today?\nHuman: files/customer-comments.csv\n\n how many rows are in this file?\nAI: There are 100 rows in the file.\n00 rows in the file.\nHuman: can you count how many rows are from the US?\nAI: There are 42 rows from the US.\n2 rows from the US.'}}


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: pandas_calculator
Action Input: {"expr": "df['comment'].str.contains('product').sum()", "csv_file_path": "files/customer-comments.csv"}    index                                       comment country
0       1                  The product arrived damaged.      US
1       2              I'm very happy with the service!  Mexico
2       3             I have a question about my order.  Canada
3       4                        The delivery was late.      US
4       5  The customer support agent was very hel

In [11]:
invoke("how many rows are positive and how many are negative sentiment?")

Input: {'input': 'how many rows are positive and how many are negative sentiment?', 'chat_history': {'history': 'Human: Hi\nAI: Hello! How can I help you today?\n How can I help you today?\nHuman: files/customer-comments.csv\n\n how many rows are in this file?\nAI: There are 100 rows in the file.\n00 rows in the file.\nHuman: can you count how many rows are from the US?\nAI: There are 42 rows from the US.\n2 rows from the US.\nHuman: how many rows include the word product?\nAI: There are 26 rows that include the word "product".\n6 rows that include the word "product".'}}


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: csv_summary_statistics
Action Input: {"csv_input_file": "files/customer-comments.csv"}{"csv_input_file": "files/customer-comments.csv"}
sample data: 
   index                                       comment country
0      1                  The product arrived damaged.      US
1      2              I'm very happy with the service!  Mexi

## Create UI for the Chatbot

Create helper functions

In [61]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def get_response(contents):
    response = "hello"
    return response

# Create text box for user input
user_input = widgets.Textarea(value='',
                              placeholder='Enter text here...',
                              description='User:',
                              rows=2,
                              layout=widgets.Layout(width='auto'))

# Create text area for conversation display
conversation_history = widgets.Textarea(value='',
                                        description='Chat:',
                                        rows=15,
                                        layout=widgets.Layout(width='auto'))

# Create button for user to click to send their message
send_button = widgets.Button(description='Send')

def send_button_clicked(b):
    # Append user input to conversation history
    conversation_history.value += 'User: ' + user_input.value + '\n'
    response = invoke(user_input.value) 
    conversation_history.value += 'Assistant: ' + response + '\n'
    
    # Clear user input box
    user_input.value = ''

send_button.on_click(send_button_clicked)



In [62]:
# Display everything
display(conversation_history, user_input, send_button)

Textarea(value='', description='Chat:', layout=Layout(width='auto'), rows=15)

Textarea(value='', description='User:', layout=Layout(width='auto'), placeholder='Enter text here...', rows=2)

Button(description='Send', style=ButtonStyle())